## Load modules

In [1]:
import xarray as xr
import numpy as np
import cdo
import copy
import matplotlib.pyplot as plt
from scipy import ndimage
from tqdm.notebook import tqdm

In [2]:
%matplotlib qt5

## Initialisation cdo

In [3]:
cdo = cdo.Cdo()

## Declare paths

In [4]:
inputpath_raw = '/data/cburgard/MELT_PARAM_NONRESOLVED/raw/'
inputpath_interim = '/data/cburgard/MELT_PARAM_NONRESOLVED/interim/'
inputpath_NEMO = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_NEMO_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_domaincfg = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/'

## Define grids

In [5]:
## source grid
fBedMachine_grid=inputpath_interim + 'basins_for_param_v5_raster_extrapolate_with_zmin_zmax_isfconc_withgrid.nc'
BedMachine_grid=xr.open_dataset(fBedMachine_grid)

## target grid
fNEMO_grid = inputpath_NEMO_interim + 'NEMO_gridT_eORCA1_cdo.nc'
ds_nemo_grid=xr.open_dataset(fNEMO_grid)
fcoordinates_file = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/coordinates.nc'
coordinates_file = xr.open_dataset(fcoordinates_file)
ds_nemo_grid_without_nan = inputpath_interim + 'coordinates_cdo.nc'

## mask of all cavities


## Inputs files

In [6]:
# BedMachine state data
ds_BedMachine=xr.open_dataset(inputpath_NEMO_interim + 'BedMachine_v3_aggregated2km_allvars_withgrid.nc')

# NEMO bedrock data
ds_nemo_file=xr.open_dataset(inputpath_NEMO + 'eORCA1.4.3_OpenSeas_Open6largestISF_ModStraights_domain_cfg_withgrid.nc').squeeze().drop_vars('time')
ds_nemo = ds_nemo_file[['isf_draft', 'bathy_metry']] #,'nav_lon_bnds','nav_lat_bnds'

In [7]:
glamt = coordinates_file['glamt']
glamt.attrs = ds_nemo['nav_lon'].attrs

gphit = coordinates_file['gphit']
gphit.attrs = ds_nemo['nav_lat'].attrs

In [8]:
ds_nemo_corr = ds_nemo.assign_coords({'nav_lon': glamt, 'nav_lat': gphit})

In [9]:
ds_nemo_corr['mask_isf'] = (ds_nemo_corr['isf_draft'] > 0).astype(float)

In [10]:
ds_nemo_corr['cell_area'] = coordinates_file['e1t'] * coordinates_file['e2t']

# compute area not resolved by NEMO
**Method from Pierre:**
- conservative interpolation of Elmer floating cell to NEMO
- set to 1 value between 0 and 1 (ie area to parametrised in NEMO)
- conservative interpolation back to ELMER to give the cell that need to be included in the param and the area scale factor (note from Clara: not sure this last step is really needed)

In [ ]:
# CDO interpolation
isfconc_NEMOgrid0 = cdo.remapbil(fNEMO_grid,input=ds_BedMachine, returnArray='isf_conc')

In [ ]:
isfconc_NEMOgrid = xr.DataArray(data=isfconc_NEMOgrid0, dims=['y','x']).assign_coords({'y': ds_nemo_grid.y, 
                                                                                       'x': ds_nemo_grid.x})

In [ ]:
isfconc_NEMOgrid.plot()

In [ ]:
NEMO_cell_to_param = (isfconc_NEMOgrid > 0) & (isfconc_NEMOgrid < 0.99)

In [ ]:
# create clean data_set array (cdo is very sensitive)
ds_elmer_sftflf = xr.Dataset(
    data_vars=dict(
        sftflf=(["time","ncells"], ds_elmer.sftflf.values),
        time_instant=(["time"],ds_elmer.time_instant.values),
        lon_bnds=(["ncells","vertices"], ds_elmer_grid.lon_bnds.values),
        lat_bnds=(["ncells","vertices"], ds_elmer_grid.lat_bnds.values),
        time_instant_bnds=(["time","axis_nbounds"],ds_elmer.time_instant_bounds.values),
    ),
    coords=dict(
        lon=(["ncells"],ds_elmer_grid.lon.values,ds_elmer_grid.lon.attrs),
        lat=(["ncells"],ds_elmer_grid.lat.values,ds_elmer_grid.lat.attrs),
    ),
)

In [ ]:
## CDO inperpolation
### cdo.remapcon(f'{target_grid}',input=input_xrdataset, returnArray=variable_name_to_process)
#NEMO_sftflf=cdo.remapcon(f'{fNEMO_grid}', input=ds_elmer_sftflf, returnArray='sftflf', env={"CDO_REMAP_NORM": "areafrac"}).squeeze()
#NEMO_sftflf[0,50:60]=1 # => specific to ORCA2

In [ ]:
# set to 1 cell to parametrised
#NEMO_cell_to_param=np.zeros(shape=NEMO_sftflf.shape)
#NEMO_cell_to_param=np.where((NEMO_sftflf > 0) & (NEMO_sftflf<=1),1,0)

**Method from Clara:**
- interpolation of NEMO on BedMachine grid
- identify cells that are not or not completely taken into account in NEMO => regions non-resolved in NEMO
- of these: check which are associated to a basin that is not on the NEMO grid and replace them
- then, compute the area scale factor

In [11]:
ds_nemo_mask = ds_nemo_corr['mask_isf']

In [12]:
isfcellconc_BMgrid0 = cdo.remapbil(fBedMachine_grid,input=ds_nemo_corr, returnArray='mask_isf')

In [13]:
isfcellconc_BMgrid = xr.DataArray(data=isfcellconc_BMgrid0, dims=['y','x']).assign_coords({'y': BedMachine_grid.y, 
                                                         'x': BedMachine_grid.x})

In [14]:
# Identify ice shelves in "reality"

In [15]:
isfconc_BM_orig = ds_BedMachine['isf_conc']

In [ ]:
isfconc_BM_orig.plot()

Assign basins to the regions of isf conc between 0 and 1

In [ ]:
ds_ANT_basins.plot()

In [16]:
## Read basin file at 2km
ds_ANT_basins=xr.open_dataset(inputpath_raw + 'basins_for_param_v5_raster_extrapolate_setgrid.nc')['Band1']

In [17]:
## Identify non resolved subregions or the whole ice shelves not present in NEMO but present in BedMachine
# sum of concentration on original and NEMO grid (if they are both > 0.99 - so 1.98, this is a fully covered cell in both)
sum_isfconc = isfconc_BM_orig + isfcellconc_BMgrid 
# all the cells that are partially covered in one of the two and where there is isf in BedMachine gets assigned a number
basin_mask_nonresolved = ((sum_isfconc < 1.98) & (isfconc_BM_orig > 0)) * ds_ANT_basins
basin_mask_nonresolved = basin_mask_nonresolved.where(basin_mask_nonresolved > 0)

In [ ]:
ds_ANT_basins.plot()

Check if all the basins are present in NEMO

In [18]:
## CDO inperpolation
NEMO_basins=cdo.remapnn(ds_nemo_grid_without_nan,input=ds_ANT_basins, returnArray='Band1').squeeze() 
#compare_NEMO_basins=cdo.remapnn(fNEMO_grid,input=ds_ANT_basins, returnArray='Band1').squeeze() #=> doesn't work because the coordinates file is not in cdo format and  

In [19]:
NEMO_basins_da = xr.DataArray(data=NEMO_basins,dims=ds_nemo_corr.dims)
#compare_NEMO_basins_da = xr.DataArray(data=compare_NEMO_basins,dims=ds_nemo_corr.dims)

In [ ]:
# check the bounds are the right ones
#(NEMO_basins_da - compare_NEMO_basins_da).where(compare_NEMO_basins_da != 6).plot(vmax=10)

In [20]:
# define basins "present" in NEMO
ocean_mask = ds_nemo_file['bathy_metry'] > 0
basin_in_ocean = NEMO_basins_da.where(ocean_mask)

In [ ]:
plt.figure()
basin_in_ocean.plot(vmin=120,vmax=122)

In [21]:
# get list of ID avalable in NEMO along the 'coastal' band
NEMO_basins_list = np.unique(basin_in_ocean).astype(int)
ANT_basins_list = np.unique(basin_mask_nonresolved).astype(int)

<ipython-input-21-b5f75a13dfc1>:2: RuntimeWarning: invalid value encountered in cast
  NEMO_basins_list = np.unique(basin_in_ocean).astype(int)
<ipython-input-21-b5f75a13dfc1>:3: RuntimeWarning: invalid value encountered in cast
  ANT_basins_list = np.unique(basin_mask_nonresolved).astype(int)


In [22]:
basins_not_in_NEMO_grid = []
for bbasin in ANT_basins_list:
    if bbasin not in NEMO_basins_list:
        print(bbasin)
        basins_not_in_NEMO_grid.append(bbasin)

54
56
121
193


In [ ]:
basin_mask_nonresolved.plot()

In [23]:
ds_ANT_basins_missing = ds_ANT_basins.where(~ds_ANT_basins.isin(basins_not_in_NEMO_grid))

In [24]:
def fill_nearest(xarr):
    """Fill NaNs using nearest-neighbor interpolation."""
    data = xarr.values
    mask = np.isnan(data)
    if not np.any(mask):
        return xarr  # No NaNs to fill

    # Get coordinates of valid (non-NaN) and NaN points
    filled = ndimage.distance_transform_edt(mask, return_distances=False, return_indices=True)
    filled_data = data[tuple(filled)]
    return xr.DataArray(filled_data, coords=xarr.coords, dims=xarr.dims, attrs=xarr.attrs)


In [25]:
# Apply to Dataset
#filled_ds = basin_mask_nonresolved_missing.copy()
filled_basins = fill_nearest(ds_ANT_basins_missing)
basin_mask_nonresolved_clean = filled_basins.where(basin_mask_nonresolved > 0)

In [ ]:
basin_mask_nonresolved_clean.plot()

In [26]:
# Double check that I now only have basins left that are there both on the BedMachine and NEMO grid
new_ANT_basins_list = np.unique(basin_mask_nonresolved_clean).astype(int)

<ipython-input-26-ed589993423e>:2: RuntimeWarning: invalid value encountered in cast
  new_ANT_basins_list = np.unique(basin_mask_nonresolved_clean).astype(int)


In [ ]:
for bbasin in new_ANT_basins_list:
    if bbasin not in NEMO_basins_list:
        print(bbasin)

In [ ]:
# Here the mask for the unresolved regions is ready! Congrats!

In [ ]:
np.isfinite(basin_mask_nonresolved_clean).plot()

In [ ]:
# Write to file (no it does not make sense because the things are not resolved...)
# NEMO_mask = np.isfinite(basin_mask_nonresolved_clean)
# NEMO_base = ds_nemo['isf_draft']*NEMO_mask
# NEMO_bathy= ds_nemo['bathy_metry']*NEMO_mask

Compute the area scale factor

In [ ]:
# will need to do a loop over all numbers in new_ANT_basins_list
# for each, compute area of unresolved area at each depth
# isfpar_area the non-resolved ice shelf area (m^2) per basin and per vertical level
# isfpar_basin the map of basin numbers
# zmin and zmax (double check how I did it before with zmin and zmax

In [27]:
mesh_mask = xr.open_dataset(inputpath_NEMO + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_mesh_mask_withgrid.nc')

In [28]:
tmask_3D = mesh_mask['tmask'].squeeze().drop('time_counter').astype(float)

In [29]:
isfcellconc_3D_BMgrid0 = cdo.remapbil(fBedMachine_grid,input=tmask_3D, returnArray='tmask')

In [ ]:
isfcellconc_3D_BMgrid0.shape

In [30]:
isfcellconc_3D_BMgrid = xr.DataArray(data=isfcellconc_3D_BMgrid0, dims=['depth','y','x']).assign_coords({'y': BedMachine_grid.y, 
                                                         'x': BedMachine_grid.x, 'depth': tmask_3D.nav_lev.values})

In [31]:
# check the depth-level dependent ice-shelf concentration
isfcellconc_3D_BMgrid.isel(depth=40).where((isfcellconc_3D_BMgrid.isel(depth=40) > 0) 
                                           & (isfcellconc_3D_BMgrid.isel(depth=40) < 1)).plot()

In [32]:
grid_cell_area = xr.open_dataset(inputpath_NEMO_interim + 'ISMIP6_grid_AIS_2000m.nc')['cell_area'] # this is not correct yet but using it in the meantime

In [33]:
area_per_depth = grid_cell_area * isfcellconc_3D_BMgrid

In [34]:
isfpar_area_list = []
for bbasin in tqdm(new_ANT_basins_list[:-1:]):
    isfpar_area_bb = area_per_depth.where(basin_mask_nonresolved_clean == bbasin, drop=True).sum(['y','x'])
    isfpar_area_list.append(isfpar_area_bb.rename({'depth': 'nav_lev'}).assign_coords({'basin': bbasin}))
isfpar_area_all = xr.concat(isfpar_area_list, dim='basin')

  0%|          | 0/130 [00:00<?, ?it/s]

ValueError: Cannot apply_along_axis when any iteration dimensions are 0

In [41]:
isfpar_area_all = xr.concat(isfpar_area_list, dim='basin')

In [47]:
isfpar_area_all.rename('area').to_netcdf(inputpath_interim + 'isfpar_129area_Clara.nc')

Define where to reinject the meltwater => maybe look at this file: prepare_eORCA1_input

In [48]:
# Define NEMO coastal mask of the area to parametrised
def get_coastal_msk(mask_in,lewp):
    nj_out=mask_in.shape[0]
    ni_out=mask_in.shape[1]

    if lewp:
        mask=np.zeros(shape=(nj_out,ni_out+2))
        mask[:,1:-1]=mask_in
        mask[:, 0]=mask[:,-2]
        mask[:,-1]=mask[:, 1]
        xslc=slice(0,ni_out)
    else:
        mask=mask_in
        xslc=slice(1,-1)
        
    mask_coast=np.zeros(shape=(nj_out,ni_out))
    mask_coast[1:-1,xslc]= ( mask[1:-1,1:-1] + 
                             mask[0:-2,1:-1] + mask[2::,1:-1] + mask[1:-1,0:-2] + mask[1:-1,2::] +
                             mask[0:-2,0:-2] + mask[2::,2::]  + mask[2::,0:-2]  + mask[0:-2,2::] ) * mask[1:-1,1:-1]
    mask_coast[(mask_coast > 1) & (mask_coast < 9)] = 10
    mask_coast[mask_coast!=10]=np.nan
    mask_coast=mask_coast.astype(np.float32)
    mask_coast[mask_coast==10]=1
    
    return mask_coast

In [ ]:
# get coastal cell along area to parametrized
# method:
# - reverse the cell_to_param mask
# - detect contour
# - pick from contour only the ocean points
NEMO_mask_param_basins=-np.zeros(shape=NEMO_cell_to_param.shape)
#NEMO_mask_param_basins=np.where(NEMO_cell_to_param>0,0,1)
NEMO_mask_param_basins=get_coastal_msk(NEMO_mask,True)
NEMO_mask_param_basins[np.isnan(NEMO_mask_param_basins) | (NEMO_mask==0)]=-1



# FROM PIERRE - NOT NECESSARILY EVERYTHING FOR ME

## Read NEMO geometry and ELMER data (I don't think that it's needed here)

In [ ]:
# Read domain_cfg.nc
#ds_nemo_domain=xr.open_dataset('domain_cfg_O2.nc')
#NEMO_isfd=ds_nemo_domain.top_level.squeeze()

# get NEMO mask
#NEMO_mask=ds_nemo_domain.bottom_level.squeeze()
#NEMO_mask=np.where(NEMO_mask>0,1,0)
#NEMO_mask[30::,:]=1 # => specific to ORCA2

# interpolation back to Elmer to retreive the area scale factor
#ds_nemo_param=copy.deepcopy(ds_nemo_grid).rename_vars({'dummy': 'cell_to_param'})
#ds_nemo_param['cell_to_param'].values=NEMO_cell_to_param.squeeze()

## CDO inperpolation
#ELMER_cell_to_param_sf=ds_elmer.sftflf.values.squeeze()

## Define the cell where to activate the param in NEMO

Maybe this need to be done before the reinterpolation back to ELMER and the draft distribution.
 * compute list of basin from the NEMO_mask_param_basins variables
 * mask the missing basin in the NEMO_basins variable
 * do a nn extrapolation on the NEMO grid
 * interpolate back to ELMER
 * compute the isf draft distribution

In [ ]:
# Define NEMO coastal mask of the area to parametrised
def get_coastal_msk(mask_in,lewp):
    nj_out=mask_in.shape[0]
    ni_out=mask_in.shape[1]

    if lewp:
        mask=np.zeros(shape=(nj_out,ni_out+2))
        mask[:,1:-1]=mask_in
        mask[:, 0]=mask[:,-2]
        mask[:,-1]=mask[:, 1]
        xslc=slice(0,ni_out)
    else:
        mask=mask_in
        xslc=slice(1,-1)
        
    mask_coast=np.zeros(shape=(nj_out,ni_out))
    mask_coast[1:-1,xslc]= ( mask[1:-1,1:-1] + 
                             mask[0:-2,1:-1] + mask[2::,1:-1] + mask[1:-1,0:-2] + mask[1:-1,2::] +
                             mask[0:-2,0:-2] + mask[2::,2::]  + mask[2::,0:-2]  + mask[0:-2,2::] ) * mask[1:-1,1:-1]
    mask_coast[(mask_coast > 1) & (mask_coast < 9)] = 10
    mask_coast[mask_coast!=10]=np.nan
    mask_coast=mask_coast.astype(np.float32)
    mask_coast[mask_coast==10]=1
    
    return mask_coast

In [ ]:
# get coastal cell along area to parametrized
# method:
# - reverse the cell_to_param mask
# - detect contour
# - pick from contour only the ocean points
NEMO_mask_param_basins=-np.zeros(shape=NEMO_cell_to_param.shape)
#NEMO_mask_param_basins=np.where(NEMO_cell_to_param>0,0,1)
NEMO_mask_param_basins=get_coastal_msk(NEMO_mask,True)
NEMO_mask_param_basins[np.isnan(NEMO_mask_param_basins) | (NEMO_mask==0)]=-1



In [ ]:
plt.pcolormesh(get_coastal_msk(NEMO_mask,True))
plt.colorbar()

In [ ]:
plt.pcolormesh(NEMO_mask_param_basins)
plt.colorbar()

# Manage basins
## Interpolation of basin variable from Elmer to NEMO grid
**Method:** cdo nearest neighbourg

**Warning** : 
 * possibly this need to be move after the domain cfg computation
 * why not use the extrapolate basin file on stereo ?

In [ ]:
## Read basin file at 2km
ds_ANT_basins=xr.open_dataset(inputpath_raw + 'basins_for_param_v5_raster_extrapolate_setgrid.nc')

## CDO inperpolation
NEMO_basins=cdo.remapnn(f'{fNEMO_grid}',input=ds_ANT_basins, returnArray='Band1').squeeze()

## Get all the basins id mask along the 'coastal' band

In [ ]:
# get basin number for each coastal cell
#NEMO_mask_param_basins=np.where((NEMO_mask_param_basins >= 0), NEMO_basins, -1)

In [ ]:
NEMO_basins

In [ ]:
mask_basins_nonresolved_points = isfcellconc_BMgrid

In [ ]:
# get list of ID avalable in NEMO along the 'coastal' band
NEMO_basins_list = np.unique(NEMO_mask_param_basins[NEMO_mask_param_basins >= 0]).astype(int)
ANT_basins_list = np.unique(ds_ANT_basins.Band1).astype(int)

## Interpolation back to Elmer from NEMO grid
**Why:** We need to do this in case some small basins have been removed during the nn interpolation to NEMO. Without this step, there is possibility that the sum of the area distribution seen by NEMO do not match the total area to parametrised in ELMER.  

In [ ]:
# Define the new basin mask based on the list of available basins
## Mask all id not present in the list in the original basin file
ANT_basins_corr=ds_ANT_basins.Band1.copy()
for basin in set(ANT_basins_list) - set(NEMO_basins_list):
    ANT_basins_corr.values[ANT_basins_corr.values == basin] = np.nan


In [ ]:

## Extrapolate the fill the corrected id mask
from scipy.interpolate import NearestNDInterpolator
data = ANT_basins_corr.values
mask=np.where(~np.isnan(data))
interp = NearestNDInterpolator(np.transpose(mask), data[mask])
ANT_basins_corr.values = interp(*np.indices(data.shape))


In [ ]:
## Interpolate the correct mask to Elmer
ELMER_basins=cdo.remapnn(f'{fELMER_grid}',input=ANT_basins_corr, returnArray='Band1')

# Interpolation back from NEMO to ELMER to retreive the area scale factor

## Compute depth distribution

In [ ]:
# get elmer element area, ice shelf draft and floating cell mask
isfd=ds_elmer.base.values.squeeze()
mask_isf=ds_elmer.sftflf.values.squeeze()
cell_area=np.float64(ds_elmer.cell_area.values.squeeze())

# get NEMO depth range
e3t=ds_nemo_domain.e3t_1d.values.squeeze()

**WARNING** : the basin list below need to match the list of basin in NEMO. Not sure it works in case some basins do not extrapolate far enough (nn approach issue)

In [ ]:
# compute weight for histogram
# As there is no melt in Elmer on the partially grounded element, these element should be excluded
ELMER_cell_to_param_sf[np.isnan(ELMER_cell_to_param_sf) | (mask_isf < 1)]=0.0
weight=np.float64(cell_area)*np.float64(ELMER_cell_to_param_sf)
ELMER_isfd=isfd*mask_isf

# define histogram bin
binbnds=np.zeros(shape=(e3t.shape[0]+1,))
binbnds[1::]=np.cumsum(e3t[:])

# retreive basin list id
basin_list=set(ELMER_basins.flatten())

# compute historgram for each basin
hist_isfd=np.zeros(shape=(len(basin_list),len(e3t)))
for ib, ibasin in enumerate(basin_list):
    hist_isfd[ib,:],_=np.histogram(-ELMER_isfd[ELMER_basins==ibasin],binbnds,weights=weight[ELMER_basins==ibasin])
    print(ib, ibasin, weight[ELMER_basins==ibasin].sum(), hist_isfd[ib,:].sum(), weight[ELMER_basins==ibasin].sum()-hist_isfd[ib,:].sum(), max(-ELMER_isfd[ELMER_basins==ibasin]), min(-ELMER_isfd[ELMER_basins==ibasin]))

In [ ]:
zmin_basin = {}
zmax_basin = {}

for ibasin in basin_list:
    # Select the cells in the current basin
    mask = (ELMER_basins == ibasin)
    valid_draft = -ELMER_isfd[mask]  # Make draft positive down
    valid_weight = weight[mask]

    # Filter out zero-weight or nan values
    valid = (valid_weight > 0) & np.isfinite(valid_draft)
    if np.any(valid):
        # Use weighted percentile function (approximation via repeat)
        sample_depths = np.repeat(valid_draft[valid], np.maximum(valid_weight[valid]/np.min(valid_weight[valid]),1).astype(int))
        zmin = np.percentile(sample_depths, 1)
        zmax = np.percentile(sample_depths, 99)
    else:
        zmin = np.nan
        zmax = np.nan

    zmin_basin[ibasin] = zmin
    zmax_basin[ibasin] = zmax


In [ ]:
zmin_map = np.full_like(NEMO_mask_param_basins, 0, dtype=np.float64)
zmax_map = np.full_like(NEMO_mask_param_basins, 0, dtype=np.float64)

for ibasin in basin_list:
    zmin_map[NEMO_mask_param_basins == ibasin] = zmin_basin[ibasin]
    zmax_map[NEMO_mask_param_basins == ibasin] = zmax_basin[ibasin]

## Define the output file

In [ ]:
# for Nico's version:
# Étape 1 : Extraire les ID uniques triés (en ignorant les valeurs invalides éventuelles comme -1)
unique_ids = np.sort(np.unique(NEMO_mask_param_basins[NEMO_mask_param_basins >= 0]))

# Étape 2 : Créer un dictionnaire de remapping avec un décalage de +1
id_map = {orig_id: new_id for new_id, orig_id in enumerate(unique_ids, start=1)}

# Étape 3 : Appliquer le remapping
NEMO_mask_param_basins_remapped = np.full_like(NEMO_mask_param_basins, fill_value=0, dtype=np.int32)  # 0 pour "non défini"
for orig_id, new_id in id_map.items():
    NEMO_mask_param_basins_remapped[NEMO_mask_param_basins == orig_id] = new_id

# Étape 4 : Nouvelle liste de basins consécutifs à partir de 1
basin_list_remapped = np.arange(1, len(unique_ids) + 1, dtype=np.int32)

In [ ]:
# Compute global min/max for each variable
zmin_global_min = float(np.nanmin(zmin_map))
zmin_global_max = float(np.nanmax(zmin_map))
zmax_global_min = float(np.nanmin(zmax_map))
zmax_global_max = float(np.nanmax(zmax_map))

area_dist_min = float(np.nanmin(hist_isfd))
area_dist_max = float(np.nanmax(hist_isfd))

param_id_min = int(np.nanmin(NEMO_mask_param_basins_remapped))
param_id_max = int(np.nanmax(NEMO_mask_param_basins_remapped))

# Create dataset for output
ds_out = xr.Dataset(
    data_vars=dict(
        isf_param_id=(["x", "y"], NEMO_mask_param_basins_remapped.astype(np.int32), dict(
            units='1',
            long_name='ice shelf parameterization basin ID',
            valid_min=param_id_min,
            valid_max=param_id_max
        )),
        
        isf_area_dist=(["z", "id"], hist_isfd.T, dict(
            units='m2',
            long_name='ice shelf draft area distribution per depth bin',
            valid_min=area_dist_min,
            valid_max=area_dist_max
        )),
        
        isf_zmin=(["x", "y"], zmin_map, dict(
            units='m',
            long_name='ice shelf minimum draft depth (10th percentile)',
            valid_min=zmin_global_min,
            valid_max=zmin_global_max
        )),
        
        isf_zmax=(["x", "y"], zmax_map, dict(
            units='m',
            long_name='ice shelf maximum draft depth (90th percentile)',
            valid_min=zmax_global_min,
            valid_max=zmax_global_max
        )),
    ),
    
    coords=dict(
        lon=(["x", "y"], ds_nemo_domain.glamt.values.squeeze(), dict(
            units='degrees_east',
            long_name='longitude'
        )),
        lat=(["x", "y"], ds_nemo_domain.gphit.values.squeeze(), dict(
            units='degrees_north',
            long_name='latitude'
        )),
        bin_width=(["z"], e3t, dict(
            units='m',
            long_name='vertical bin thickness'
        )),
        basin=(["id"], np.array(list(basin_list_remapped)).astype(np.int32), dict(
            long_name='list of unique basin IDs'
        )),
    ),
)

# Save to NetCDF
ds_out.to_netcdf('isf_param.nc')


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# Load your dataset
ds = xr.open_dataset('isf_param.nc')

# Extract data
param_id = ds['isf_param_id']
lon = ds['lon']
lat = ds['lat']

# Identifier les ID valides (1 à 84)
valid_ids = np.arange(1, 85)

# Créer une permutation aléatoire des ID
shuffled_ids = np.random.permutation(valid_ids)

# Construire un dictionnaire de remappage
remap_dict = dict(zip(valid_ids, shuffled_ids))

# Appliquer le remappage sur une copie
param_id_shuffled = np.full_like(param_id, fill_value=-1)
for old_id, new_id in remap_dict.items():
    param_id_shuffled[param_id == old_id] = new_id


# Mask bad values (-1)
param_id_masked = np.ma.masked_where(param_id.values == 0, param_id_shuffled)

# Set up projection
proj = ccrs.SouthPolarStereo()  # EPSG:3031

# Create figure and axis
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=proj)
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())

# Add land and ice shelves using Natural Earth features
land = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                    edgecolor='black', facecolor='lightgray')
ice_shelves = cfeature.NaturalEarthFeature('physical', 'antarctic_ice_shelves_polys', '50m',
                                           edgecolor='blue', facecolor='none')

ax.add_feature(land, zorder=1)
ax.add_feature(ice_shelves, zorder=2)

# Plot the param_id map
c = ax.pcolormesh(lon, lat, param_id_masked,
                  transform=ccrs.PlateCarree(),
                  cmap='tab20', shading='auto', zorder=0)

# Add colorbar
cb = plt.colorbar(c, ax=ax, orientation='horizontal', pad=0.05, shrink=0.7)
cb.set_label(param_id.long_name)

# Add gridlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
gl.top_labels = False
gl.right_labels = False

plt.title('Ice Shelf Parameterization Basin ID (ORCA2))')
#plt.tight_layout()
#
fig.savefig('basin_id_map.png', dpi=150, bbox_inches='tight', transparent=False)